# ONLINE GBM RESPONSE GENERATOR

This is a tool that allows your to generate RSP and RSP2 files for GBM triggered and daily data.
It relies on two packages: [gbmgeometry]() and [gbm_drm_gen](). 

Following the instructions below, simply input your desired parameters and run the notebook. 


You responses will be available in the root of the notebook server as `output.tar` when you are finished.

![alt text](https://raw.githubusercontent.com/grburgess/gbm_drm_gen/master/logo.png)

In [ ]:
from pathlib import Path
import tarfile


import matplotlib.pyplot as plt
%matplotlib notebook

from threeML import download_GBM_trigger_data, download_GBM_daily_data

import gbmgeometry
from gbmgeometry.io.data_download import download_posthist, download_trigdat, get_official_location

gbm_detectors = [f"n{x}" for x in range(10)]
gbm_detectors.extend(['na','nb','b0','b1'])


import gbm_drm_gen




# Setup the variables

Here you will decide if you use either **triggered** or **daily** data. Then you will enter the variables for the trigger or day.


## Type of data

Run only one section as it sets up the rest of the notebook. 


### Trigger data

This if for triggered data such as GRBs. You need to know the `burst number` and set it below. As an example, GRB080916C is currently entered.
Once you have set the burst number, run both cells below. Some data will be downloaded which might take a few moments.

In [ ]:
grb_trigger_number = "bn080916009"

# do not alter
using_daily_data = False

In [ ]:

download_trigdat(grb_trigger_number)
spacecraft_file = list(Path('.').glob(f'trigdat_{grb_trigger_number}_v*.h5'))[0]

gbm_data = download_GBM_trigger_data(grb_trigger_number, cspec_only=True)

for k,v in gbm_data.items():
    
    Path(v['rsp']).unlink()
    
    v.pop('rsp')

### Daily data

This is for GBM daily data. This is useful for untriggered files. You will need to know the fermi MET of the data to set a `T0` for the responses. This makes the time relative to that `T0` rather than the large MET numbers native to the time-axis.

Specify the year, month and day as shown below. Then run the two cells. Some data will be downloaded which might take a moment. 


In [ ]:
#year
year = '20'
month = '01'
day = '16'

# MET to zero the daily data
# Use None if you do not want to zero
T0 = 600835545.0

using_daily_data = True

In [ ]:
download_posthist(year, month, day)
spacecraft_file = f"poshist_{year}{month}{day}.h5"


gbm_data = download_GBM_daily_data(year, month, day, cspec_only=True)

## Sky location

Now we will set the sky location (RA, Dec) that the responses will be created for. You can set the official location from the FSSC for GRBs or triggered data, or you can set a custom location.



### Official location

This checks the FSSC for the cataloged location of the GRB or trigger.


In [ ]:
ra, dec, loc_err = get_official_location(grb_trigger_number)
print(f"rs:{ra}, dec:{dec}, err:{loc_err}")

### Custom location

Here you can set a custom location in degrees.


In [ ]:
ra = 10.
dec = 10

# Examine geometry

We can now have a look at the location in Fermi spacecraft coordinates. You can use the position interpolator to examine many things about Fermi for the current time. Check out the documentation for more details.


In [ ]:
if using_daily_data:
    
    position_interpolator = gbmgeometry.PositionInterpolator.from_poshist_hdf5(spacecraft_file, T0=T0)

else:
    
    position_interpolator = gbmgeometry.PositionInterpolator.from_trigdat_hdf5(spacecraft_file)


## Spacecraft blockage

Here, you can plot a 3D model of Fermi and see which detectors are blocked by other parts of the spacecraft. Green lines indicate the rays coming from the input position, and red dots indicate blockage for that ray. You can use you mouse to move the spacecraft.


In [ ]:
from gbmgeometry.spacecraft.fermi import *
from astropy.coordinates import SkyCoord

coord = SkyCoord(ra=ra,dec=dec, unit='deg', frame='icrs')



fermi = Fermi(quaternion=position_interpolator.quaternion(0) , sc_pos=position_interpolator.sc_pos(0))

fermi.add_ray(ray_coordinate=coord)

fermi.plot_fermi(color_dets_different=True, 
                 plot_det_label=False,
                 with_intersections=True, 
                 with_rays=True);

# Generate Responses

Now you can generate the responses. You can to make a single response for a given time or create RSP2s which have multiple responses that account for the changing position of the Fermi spacecraft during the time range. 


## Single Epoch Response

This generates responses for each detector on GBM at a given time relative to the T0 or trigger time specified above.



In [ ]:
time = 0. # time relative to T0 or trigger time.

In [ ]:
with tarfile.open("output.tar", "w") as tar:
    for detector in gbm_detectors:

        if using_daily_data:

            drm_generator = gbm_drm_gen.DRMGen.from_128_bin_data(det_name= detector,
                                                                 T0=T0,
                                                                 time=time, 
                                                                 poshist = spacecraft_file,
                                                                 mat_type = 2, # direct response + atmospheric scattering
                                                                 cspecfile = gbm_data[detector]['cspec'])
            
            stub = f"bn{year}{month}{day}"
            
            

        else:

            drm_generator = gbm_drm_gen.DRMGen.from_128_bin_data(det_name= detector,
                                                     time=time,
                                                     trigdat = spacecraft_file,
                                                     mat_type = 2, # direct response + atmospheric scattering
                                                     cspecfile = gbm_data[detector]['cspec'])
            
            stub = f"{grb_trigger_number}"


        
        file_name = f"{stub}_{detector}.rsp"
        
        drm_generator.to_fits(ra=ra, dec=dec, filename=f"{stub}", overwrite=True)
        
        tar.add(file_name)

# clean up
_ =[x.unlink() for x in Path('.').glob('*.rsp')]

## Time resolved response (RSP2)

This generates RSP2 which contain multiple responses at a given temporal cadance over the time range you specify. When using 3ML to do spectral analysis, 3ML will weight these responses to try and account for the slew of the spacecraft with time.



### Define the times

Now define the time range for the RSP2s to be valid. This assumes you specified a `T0` in MET if you are using daily data.
Times are relative to T0 or the GRB trigger time. 

In [ ]:
tstart = 0
tstop = 10
delta_time = 2 # the time spacing between intervals

### Produce the responses

In [ ]:

with tarfile.open("output.tar", "w") as tar:
    

    for detector in gbm_detectors:

        if using_daily_data:

            drm_generator = gbm_drm_gen.DRMGen.from_128_bin_data(det_name= detector,
                                                                 T0=T0,
                                                                 time=0, # time relative to T0 or trigger time.
                                                                 poshist = spacecraft_file,
                                                                 mat_type = 2, # direct response + atmospheric scattering
                                                                 cspecfile = gbm_data[detector]['cspec'])
            
            stub = f"bn{year}{month}{day}"

        else:

            drm_generator = gbm_drm_gen.DRMGen.from_128_bin_data(det_name= detector,
                                                     time=0, # time relative to T0 or trigger time.
                                                     trigdat = spacecraft_file,
                                                     mat_type = 2, # direct response + atmospheric scattering
                                                     cspecfile = gbm_data[detector]['cspec'])
            
            stub = f"{grb_trigger_number}"

        file_name = f"{stub}_{detector}.rsp2"

        gbm_drm_gen.create_rsp2(file_name,
                                response_generator=drm_generator,
                                ra=ra,
                                dec=dec,
                                tstart=tstart,
                                tstop=tstop,
                                delta_time=delta_time,
                                overwrite=True
               )

        tar.add(file_name)
_ =[x.unlink() for x in Path('.').glob('*.rsp2')]

# Download the files

Now just simply download the files by clicking over to the root or tree of the jupyter session and downloading the `output.tar` file.

**Have a nice day.**
